In [2]:
import pyhmy 
from pyhmy import (
    blockchain,
    rpc,
    account,
    staking,
    cli
)
import os
import re
from datetime import datetime
from os import path
import json
import pandas as pd
from collections import defaultdict
import requests
from threading import Thread

In [10]:
endpoint = ['https://api.s0.t.hmny.io/', 'https://api.s1.t.hmny.io/', 'https://api.s2.t.hmny.io/', 'https://api.s3.t.hmny.io/']

In [8]:
def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json; charset=utf8'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    try:
        r = requests.post(url, headers=headers, data = json.dumps(data))
    except requests.exceptions.ConnectionError as e:
        print("Error: connection error")
        time.sleep(5)
        return None
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        return None
    try:
        r.json()
    except ValueError:
        print("Error: Unable to read JSON reply")
        return None
    return r.json()
    

def getTransactionsHistory(shard, address, page, size):
    url = endpoint[shard]
    method = 'hmyv2_getTransactionsHistory'
    params = [{
        "address": address,
        "pageIndex": page,
        "pageSize": size,
        "fullTx": True,
        "txType": "ALL",
        "order": "ASC"
    }]
    return get_information(url, method, params)['result']['transactions']




In [4]:
def download_cli(path):
    env = cli.download()
    cli.environment.update(env)
    path = os.getcwd() + "/bin/hmy"
    cli.set_binary(path)

In [5]:
path = os.getcwd() + "/bin/hmy"
download_cli(path)
pyhmy_version = cli.get_version()
print(f"CLI Version: {pyhmy_version}")
version_str = re.search('version v.*-', pyhmy_version).group(0).split('-')[0].replace("version v", "")
assert int(version_str) >= 321

Saved harmony binary to: `/home/ubuntu/jupyter/harmony-log-analysis/projects/HRC_holder/bin/hmy`
CLI Version: Harmony (C) 2020. hmy, version v399-a30c33c (sj@harmony.one 2020-07-09T10:37:26+0000)


In [59]:
getTransactionsHistory(0, address, 0, 1000)

[{'blockHash': '0xc10a9de2ead070727209da365ec0dbbe6d81fc62aabba520d023d83135ceea53',
  'blockNumber': 3452696,
  'from': 'one10t79u5m9axmhs6uuzh6lu7unfdvuszjnjdxzsc',
  'timestamp': 1590444824,
  'gas': 210000,
  'gasPrice': 10000000000,
  'hash': '0x77a18b01b4933222660f6b77f81604bb62ca354c7c39604798e4723e283337e9',
  'input': '0x40c10f190000000000000000000000004cb79e605a2b3369b7b7683091adb0b41a0fee840000000000000000000000000000000000000000000000000000000000000000',
  'nonce': 23,
  'to': 'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr',
  'transactionIndex': 0,
  'value': 0,
  'shardID': 0,
  'toShardID': 0,
  'v': '0x25',
  'r': '0x554db498c628690c68aaba0ee93fa783169700d5c3ce4a127eb3a6c670d395ba',
  's': '0x635d425a8bfb555142489dddbc822cc8eb4eeb9993f5ea17b4704624425ba7e6'},
 {'blockHash': '0x5c9306166f42414f22c2cc7caf80ffa1b609bf5988d806f330d062bc22683ad5',
  'blockNumber': 3452721,
  'from': 'one13djyt8pdccvygz0g44jhch03p5tjwf5gr5l7c9',
  'timestamp': 1590445045,
  'gas': 210000,
  'gas

In [35]:
address = 'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr'
# account.get_transaction_history(address, page=0, page_size=10000, include_full_tx=True, tx_type='ALL',
#         order='ASC', endpoint=endpoint[0]
#     )
res = getTransactionsHistory(0, address)

In [44]:
address = 'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr'
transactions = []
thread_lst = defaultdict(list)
for shard in range(len(endpoint)):
    res = getTransactionsHistory(shard, address)
    transactions.extend(res['result']['transactions'])


In [46]:
len(transactions)

1771

In [ ]:
addr_set = set()
for i in range(len(transactions)):
    thread_lst[i%20].append(i)

In [39]:
def collect_data(x):
    for i in thread_lst[x]:
        global addr_set
        txs = transactions[i]
        data = txs['input']
        msg_type = data[:10]
        if msg_type != '0xa9059cbb':
            continue
        addr_bytes = data[10:74]
        addr = cli.single_call("hmy utility addr-to-bech32 {}".format(addr_bytes)).strip("\n")
        addr_set.add(addr)

threads = []
for x in range(20):
    threads.append(Thread(target = collect_data, args = [x]))
for t in threads:
    t.start()
for t in threads:
    t.join()

In [41]:
len(addr_set)

174

In [56]:
info = {0:0, 1:0, 2:0, 3:0}
prev = dict.fromkeys(list(addr_set),info)


In [65]:
prev_count = 1700
page = max(0, (prev_count-1))//1000
size = max(0, (prev_count-1))%1000+1

In [78]:
transactions = dict()

In [79]:
addr = 'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr'
prev[addr] = info
shard = 0
transactions[addr] = []

In [81]:
prev_count = prev[addr][shard]
page = max(0, (prev_count-1))//1000
size = max(0, (prev_count-1))%1000+1
res = getTransactionsHistory(shard, addr, page, 1000)

if len(res) != 0 and len(res) != size:
    transactions[addr].extend(res[size:len(res)])
    while len(res) == 1000:
        page += 1
        res = getTransactionsHistory(shard, addr, page, 1000)
        transactions[addr].extend(res[0:len(res)])
    prev[addr][shard] = page*1000+len(res)

In [82]:
prev[addr][shard]

1772

In [83]:
len(res)

772

In [87]:
prev_count = prev[addr][shard]
page = max(0, (prev_count-1))//1000
size = max(0, (prev_count-1))%1000+1
res = getTransactionsHistory(shard, addr, page, 1000)

In [88]:
len(res) == size

True

In [50]:
import pickle
pkl_file = path.join("address",'address_HRC.pkl')
if path.exists(pkl_file):
    # restart from last records
    with open(pkl_file, 'rb') as f:
        address = pickle.load(f)

In [51]:
address

{'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr': {'one10jh487kw0whnpm5a3yxrwfz3ewudeg6ju54wsc',
  'one10rr5n6qvjxlyhrhnkzxtjslzffrp4trl3wtsuh',
  'one10t79u5m9axmhs6uuzh6lu7unfdvuszjnjdxzsc',
  'one10u940w5lpe5a07j0dma6flj63zucy7j54jdj6j',
  'one10w2678a6ud4jxle62de2e7tlvmtantc9q6l7ma',
  'one1228twrqjkdqwcv2qslzytpqm7rxqw95h3memq4',
  'one122fz55g0xawv2v4kajp3qf388jzkfw40dzdgar',
  'one127la8ga6u7xh9n9qtagu5k45gq673v9wwz8lx3',
  'one127vappgzf8ku28uep2fcvjkrhvtc49rj8ufejn',
  'one12j9akvwn6e6qq8ru7nxkyyj7jexesuv883kftf',
  'one12jzn57j3n8z53nyujjy9v47ylw3v862u98wxyh',
  'one12kzpq753nzcnqy80xdzg3f5rfmjgx9jyu80k0r',
  'one12ljv0jr7duaue9ywqf09xq6kxsxk8endzz9rq2',
  'one12lv3rnznvefzx4ej8gtp342ja3tj2vgrule6ds',
  'one12pmfc8hxaulp6hppjql42dx4wk9jsg0cpqs7pe',
  'one12uujccq3eefatu4cjdqpqxldlqg90vnmd3zfkm',
  'one130d8hfdqls45qv5937pszl7pla3094tkgluhe2',
  'one134t2ky6m832cudjtq7g6g0xzfgk7krc329dwtx',
  'one137a9jtvf638cv9t49lj7ald72dzeu6whak3txm',
  'one1396xnj3chgfua3mssh5szu7lqghawsfm8lu

In [44]:
with open(pkl_file, 'wb') as f:
    pickle.dump(address, f)

In [52]:
import pickle
pkl_file = path.join("address",'txs_page_info.pkl')
if path.exists(pkl_file):
    # restart from last records
    with open(pkl_file, 'rb') as f:
        prev = pickle.load(f)

In [53]:
prev

{'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr': {0: 9237, 1: 0, 2: 0, 3: 0},
 'one1h4z5vzhgl7h223teecakpt4n6tjh864zksax8f': {0: 9, 1: 0, 2: 0, 3: 0},
 'one194rafykqj7q5x9cuk4mjyjlrn2salawwvj4uz6': {0: 0, 1: 0, 2: 0, 3: 0}}

In [18]:
addr = 'one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr'
shard = 0
prev_count = prev[addr][shard]
page = max(0, (prev_count-1))//1000
size = max(0, (prev_count-1))%1000+1
res = getTransactionsHistory(shard, addr, page, 1000)

In [19]:
len(res)

1000

In [20]:
size

1000

In [56]:
addr = 'one1h4z5vzhgl7h223teecakpt4n6tjh864zksax8f'
res = getTransactionsHistory(0, addr, 0, 1000)

In [57]:
res

[{'blockHash': '0xfdaf8487f29483aa1ff65e9120dfdb76114a316c61f2091985774574d36f3d43',
  'blockNumber': 1528449,
  'from': 'one18t4yj4fuutj83uwqckkvxp9gfa0568uc48ggj7',
  'timestamp': 1574452915,
  'gas': 3321900,
  'gasPrice': 1000000000,
  'hash': '0xa97ef319a4aad817eedf402a6eb5333f5597798b3ee2a1174c924f3d7999991b',
  'input': '0xfdacd5760000000000000000000000000000000000000000000000000000000000000001',
  'nonce': 74,
  'to': 'one1h4z5vzhgl7h223teecakpt4n6tjh864zksax8f',
  'transactionIndex': 0,
  'value': 0,
  'shardID': 0,
  'toShardID': 0,
  'v': '0x26',
  'r': '0x15eb6de36de6e5e6e01863b5f32e8716837e99ed3308f6772a34449963b601d3',
  's': '0x7e1a0772937a17eb61c177364765dc206e3408008a12770bd6fc9263785f599f'},
 {'blockHash': '0x26a326c67c0a254352e134b2ff93bc99ca3861ffcac81a7c5000e321d3c32eef',
  'blockNumber': 1528451,
  'from': 'one18t4yj4fuutj83uwqckkvxp9gfa0568uc48ggj7',
  'timestamp': 1574452931,
  'gas': 3321900,
  'gasPrice': 1000000000,
  'hash': '0x36aa1d61ef7096506ea47ca845ffd9

In [2]:

address = ['one10y76c0kyj6d9hmngf0859yx49l4c75d5z77zxr']
length = len(address)
addr_dict = dict(zip(list(range(length)), address))

In [3]:
len(addr_dict)

1